# PHMSA Company Hierarchy Analysis

**Version 1.0** - Production-Ready LLM-Based Hierarchy Identification

Automatically identifies corporate parent-subsidiary relationships in PHMSA pipeline operator data:

**Features:**
- 🔍 **Multi-Search Strategy**: 3 targeted web searches per company (DuckDuckGo)
- 🤝 **Joint Venture Detection**: Identifies JVs, ownership percentages, and primary operators
- 📅 **Recency Validation**: Prioritizes 2024-2026 ownership information
- ✅ **PHMSA Constraint**: Validates parents exist in dataset; captures external parents separately
- 📊 **Graph Resolution**: Computes ultimate parents and full hierarchy chains

**Runtime**: ~2-4 hours for 1000 companies on Databricks Serverless

In [ ]:
# Cell 1: Install Dependencies
# Run this first, then restart Python

%pip install -U langchain-community langchain-core langchain duckduckgo-search ddgs pandas networkx
dbutils.library.restartPython()

In [ ]:
# Cell 2: Initialize LLM and Search Tool

from langchain_community.chat_models import ChatDatabricks
from langchain_community.tools import DuckDuckGoSearchResults

print("Initializing LLM and search tool...")

# Initialize Claude 3.5 Sonnet via Databricks
llm = ChatDatabricks(
    endpoint="databricks-claude-sonnet-4-5",
    max_tokens=1000,
    temperature=0.1
)

# Initialize DuckDuckGo search
search_tool = DuckDuckGoSearchResults(num_results=5)

print("✓ LLM and search tool initialized")

In [ ]:
# Cell 3: Import Hierarchy Analysis Package

import sys
sys.path.append('/Workspace/Repos/YOUR_USERNAME/phmsa-company-hierarchy/')  # UPDATE THIS PATH

from phmsa_hierarchy import AgentLLMValidator, HierarchyGraphBuilder

# Initialize components
llm_validator = AgentLLMValidator(llm, search_tool)
graph_builder = HierarchyGraphBuilder()

print("✓ Agent validator and graph builder initialized")

In [ ]:
# Cell 4: Load PHMSA Data

# Load your PHMSA data from Unity Catalog
companies_df = spark.read.table("gshen_catalog.enbridge_sr_workshop.annual_hazardous_liquid_2024_updated")

# Select required columns
companies_df = companies_df.select(
    "OPERATOR_ID",
    "PARTA2NAMEOFCOMP",
    "PARTA4STREET",
    "PARTA4CITY",
    "PARTA4STATE"
).distinct()

print(f"Loaded {companies_df.count()} unique companies")
display(companies_df.limit(10))

In [ ]:
# Cell 5: Prepare Company List for Validation

# Collect all company names (parent must exist in this list)
# Note: We'll pass this directly in the UDF (no broadcast in serverless)
all_companies = [row["PARTA2NAMEOFCOMP"] for row in companies_df.select("PARTA2NAMEOFCOMP").distinct().collect()]

print(f"✓ Prepared list of {len(all_companies)} companies for validation")

In [ ]:
# Cell 6: Define Pandas UDF for Agent-Based Validation

from pyspark.sql.functions import pandas_udf, struct, col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType
import pandas as pd

# Define output schema
output_schema = StructType([
    StructField("OPERATOR_ID", IntegerType(), True),
    StructField("ORIGINAL_NAME", StringType(), True),
    StructField("IMMEDIATE_PARENT", StringType(), True),
    StructField("LLM_SEARCH_PARENT", StringType(), True),  # Raw LLM result before validation
    StructField("IS_JV", BooleanType(), True),  # Joint venture flag
    StructField("JV_PARTNERS", StringType(), True),  # Comma-separated list of JV partners
    StructField("PRIMARY_OPERATOR", StringType(), True),  # Who operates the JV
    StructField("CONFIDENCE", IntegerType(), True),
    StructField("REASONING", StringType(), True),
    StructField("ACQUISITION_DATE", StringType(), True),
    StructField("RECENT_CHANGE", BooleanType(), True),
    StructField("PARTA4STREET", StringType(), True),
    StructField("PARTA4CITY", StringType(), True),
    StructField("PARTA4STATE", StringType(), True),
])

@pandas_udf(output_schema)
def find_parent_agent(data: pd.DataFrame) -> pd.DataFrame:
    """
    Agent-based parent company finder using multi-search + LLM analysis.
    Processes a batch of companies in parallel.
    """
    from langchain_community.chat_models import ChatDatabricks
    from langchain_community.tools import DuckDuckGoSearchResults
    import sys
    sys.path.append('/Workspace/Repos/YOUR_USERNAME/phmsa-company-hierarchy/')  # UPDATE THIS
    from phmsa_hierarchy import AgentLLMValidator
    
    # Initialize for this worker
    llm_local = ChatDatabricks(
        endpoint="databricks-claude-sonnet-4-5",
        max_tokens=1000,
        temperature=0.1
    )
    search_tool_local = DuckDuckGoSearchResults(num_results=5)
    validator = AgentLLMValidator(llm_local, search_tool_local)
    
    # Get available companies from the global variable (serverless compatible)
    # Note: all_companies is captured from the outer scope
    validator.set_available_companies(all_companies)
    
    results = []
    
    for idx, row in data.iterrows():
        company_name = row["PARTA2NAMEOFCOMP"]
        operator_id = row["OPERATOR_ID"]
        address = f"{row['PARTA4STREET']}, {row['PARTA4CITY']}, {row['PARTA4STATE']}"
        
        print(f"\n{'='*60}")
        print(f"🤖 Processing [{idx+1}/{len(data)}]: {company_name}")
        print(f"{'='*60}")
        
        try:
            # Use agent to find parent
            result = validator.validate_direct(
                company_name=company_name,
                operator_id=operator_id,
                address=address
            )
            
            # Convert JV partners list to comma-separated string
            jv_partners = result.get("jv_partners")
            jv_partners_str = ", ".join(jv_partners) if jv_partners else None
            
            # Get primary operator (uppercase if present)
            primary_operator = result.get("primary_operator")
            if primary_operator:
                primary_operator = primary_operator.upper()
            
            results.append({
                "OPERATOR_ID": operator_id,
                "ORIGINAL_NAME": company_name,
                "IMMEDIATE_PARENT": result["parent"],
                "LLM_SEARCH_PARENT": result.get("llm_search_parent", result["parent"]),
                "IS_JV": result.get("is_jv", False),
                "JV_PARTNERS": jv_partners_str,
                "PRIMARY_OPERATOR": primary_operator,
                "CONFIDENCE": result["confidence"],
                "REASONING": result["reasoning"],
                "ACQUISITION_DATE": result.get("acquisition_date"),
                "RECENT_CHANGE": result.get("recent_change", False),
                "PARTA4STREET": row["PARTA4STREET"],
                "PARTA4CITY": row["PARTA4CITY"],
                "PARTA4STATE": row["PARTA4STATE"],
            })
            
            jv_flag = " [JV]" if result.get("is_jv") else ""
            operator_flag = f" operated by {primary_operator}" if primary_operator else ""
            print(f"✅ Result: {result['parent']}{jv_flag}{operator_flag} (confidence: {result['confidence']})")
            
        except Exception as e:
            print(f"❌ Error processing {company_name}: {str(e)}")
            results.append({
                "OPERATOR_ID": operator_id,
                "ORIGINAL_NAME": company_name,
                "IMMEDIATE_PARENT": "ERROR",
                "LLM_SEARCH_PARENT": "ERROR",
                "IS_JV": False,
                "JV_PARTNERS": None,
                "PRIMARY_OPERATOR": None,
                "CONFIDENCE": 0,
                "REASONING": f"Error: {str(e)}",
                "ACQUISITION_DATE": None,
                "RECENT_CHANGE": False,
                "PARTA4STREET": row["PARTA4STREET"],
                "PARTA4CITY": row["PARTA4CITY"],
                "PARTA4STATE": row["PARTA4STATE"],
            })
    
    return pd.DataFrame(results)

print("✓ Agent-based UDF defined")

In [ ]:
# Cell 7: Run Agent-Based Analysis (This takes 2-4 hours for 1000 companies)

print("Starting agent-based parent company analysis...")
print(f"Processing {companies_df.count()} companies")
print("Agent will automatically search and reason for each company")
print("\nThis will take approximately 2-4 hours. Progress will be shown below.")
print("="*60)

# Apply agent-based validation
parent_mappings_df = companies_df.select(
    find_parent_agent(
        struct(
            "OPERATOR_ID",
            "PARTA2NAMEOFCOMP",
            "PARTA4STREET",
            "PARTA4CITY",
            "PARTA4STATE"
        )
    ).alias("result")
).select("result.*")

print(f"\n{'='*60}")
print(f"✓ Processed {parent_mappings_df.count()} companies")
print(f"{'='*60}\n")

# Show sample results
display(parent_mappings_df.limit(20))

In [ ]:
# Cell 8: Build Hierarchy Graph

print("Building hierarchy graph...")

# Convert to pandas for graph building
parent_mappings_pd = parent_mappings_df.toPandas()

# Rename columns for graph builder
parent_mappings_pd = parent_mappings_pd.rename(columns={
    "ORIGINAL_NAME": "child",
    "IMMEDIATE_PARENT": "parent"
})

# Build hierarchy graph
hierarchy_df = graph_builder.build(parent_mappings_pd)

print(f"✓ Built hierarchy graph with {len(hierarchy_df)} companies")

# Show sample hierarchy results
print("\nSample hierarchy paths:")
print(hierarchy_df[['company', 'ultimate_parent', 'hierarchy_path', 'hierarchy_depth']].head(10))

In [ ]:
# Cell 9: Get Statistics

from pyspark.sql.functions import col, count, avg

print("="*60)
print("ANALYSIS STATISTICS")
print("="*60)

# Parent distribution
print("\n1. Parent Company Distribution:")
parent_dist = parent_mappings_df.groupBy("IMMEDIATE_PARENT").count().orderBy(col("count").desc())
display(parent_dist.limit(20))

# Confidence distribution
print("\n2. Confidence Score Distribution:")
confidence_stats = parent_mappings_df.select(
    avg("CONFIDENCE").alias("avg_confidence"),
    count("*").alias("total_companies")
).collect()[0]
print(f"   Average Confidence: {confidence_stats['avg_confidence']:.2f}")
print(f"   Total Companies: {confidence_stats['total_companies']}")

# Recent changes
print("\n3. Recent Ownership Changes (2024-2026):")
recent_changes = parent_mappings_df.filter(col("RECENT_CHANGE") == True)
print(f"   Companies with recent changes: {recent_changes.count()}")
display(recent_changes.select("ORIGINAL_NAME", "IMMEDIATE_PARENT", "ACQUISITION_DATE", "REASONING").limit(10))

# LLM vs Validated comparison
print("\n4. LLM Search Results vs Validated Parents:")
validation_comparison = parent_mappings_df.filter(
    (col("LLM_SEARCH_PARENT") != col("IMMEDIATE_PARENT")) & 
    (col("LLM_SEARCH_PARENT") != "ULTIMATE")
)
print(f"   Cases where LLM found parent not in PHMSA: {validation_comparison.count()}")
display(validation_comparison.select("ORIGINAL_NAME", "LLM_SEARCH_PARENT", "IMMEDIATE_PARENT", "REASONING").limit(10))

# Joint Ventures
print("\n5. Joint Ventures (JVs):")
jv_companies = parent_mappings_df.filter(col("IS_JV") == True)
print(f"   Companies identified as JVs: {jv_companies.count()}")
display(jv_companies.select("ORIGINAL_NAME", "IMMEDIATE_PARENT", "JV_PARTNERS", "PRIMARY_OPERATOR", "REASONING").limit(10))

# JVs with identified operators
jv_with_operators = parent_mappings_df.filter((col("IS_JV") == True) & (col("PRIMARY_OPERATOR").isNotNull()))
print(f"\n   JVs with identified primary operator: {jv_with_operators.count()}")
if jv_with_operators.count() > 0:
    display(jv_with_operators.select("ORIGINAL_NAME", "JV_PARTNERS", "PRIMARY_OPERATOR").limit(10))

# Ultimate parent distribution
print("\n6. Ultimate Parent Distribution:")
hierarchy_spark_df = spark.createDataFrame(hierarchy_df)
ultimate_parent_dist = hierarchy_spark_df.groupBy("ultimate_parent").count().orderBy(col("count").desc())
display(ultimate_parent_dist.limit(20))

print("\n" + "="*60)

In [ ]:
# Cell 10: Review Agent Reasoning (Quality Check)

print("Sample agent reasoning for quality assurance:")
print("="*60)

# Show examples of high confidence results
high_conf = parent_mappings_df.filter(col("CONFIDENCE") >= 8).limit(5)
print("\nHigh Confidence Results (8-10):")
for row in high_conf.collect():
    print(f"\n{row['ORIGINAL_NAME']} → {row['IMMEDIATE_PARENT']}")
    print(f"Confidence: {row['CONFIDENCE']}")
    print(f"Reasoning: {row['REASONING'][:200]}...")

# Show examples of low confidence results (may need review)
low_conf = parent_mappings_df.filter(col("CONFIDENCE") <= 5).limit(5)
print("\n" + "="*60)
print("Low Confidence Results (≤5) - May Need Review:")
for row in low_conf.collect():
    print(f"\n{row['ORIGINAL_NAME']} → {row['IMMEDIATE_PARENT']}")
    print(f"Confidence: {row['CONFIDENCE']}")
    print(f"Reasoning: {row['REASONING'][:200]}...")

print("\n" + "="*60)